In [ ]:
import warnings
warnings.simplefilter("ignore")
from sys import exit
import os.path
import os
import numpy as np
import pandas as pd
from tkinter.filedialog import askdirectory
from scipy import ndimage
from skimage import measure, io, exposure, morphology
#import matplotlib.pyplot as plt
#import seaborn as sns


: 

In [2]:
pixelSize = 101.4 #pixel size current camera in nm/pix-el: MD: 114, Andor: 128, Sam: 101.4
threshold_value = 20 #threshold value for detection, higher is more stringent
footprint = np.ones([5,5]) # footprint of gaussian
crop = [50,1120,150,1600] #crop to clean outer edges of the image
detright = 1650 # where to pull edge (uselesss)
moveval = 5 # how far to move in thr when testing
fewest = 10 # fewest objects you want to find
most = 1000 # most objects you want to find
#dirtouse = 'E:/Images/2023/6-7-23 3 ng per ul T7 with 5 mM spd condensates in well gel/sequence' # overdirectory to search for image sets

In [3]:
def vertfilt(img):
    edge = img[:,detright:]
    verts = [np.mean(x) for x in edge]
    vmax = np.max(verts)
    vmin = np.min(verts)
    veimg = np.array([(img[x]/verts[x]*vmax) for x in range(len(img))])
    return(veimg)

In [4]:
def findBackground(img):
    len_x = img.shape[0]
    len_y = img.shape[1]
    z = img.flatten()
    z = z.reshape((len(z),1))
    y = np.array([ i for i in range(len_y) ]*len_x)
    x = np.array([ [i]*len_y for i in range(len_x)  ])
    x = x.flatten()
    xx = np.power(x, 2)
    yy = np.power(y, 2)
    xy = x*y
    ones = np.ones(len(z))
    H = np.array([ xx, yy, xy, x, y, ones ])
    H = H.T
    c = np.dot(H.T, H)
    c = np.linalg.inv(c) 
    c = np.dot(c, H.T)
    c = np.dot(c, z)
    z_fit = np.dot(H, c)
    z_fit = z_fit.reshape((len_x, len_y)).astype(np.uint16)
    return(z_fit)

In [5]:
def findObjects(img, thrval):
    mean = np.mean(img)
    saliency = np.power(img - mean,2)
    dx = ndimage.sobel(saliency, 0)  # horizontal derivative
    dy = ndimage.sobel(saliency, 1)  # vertical derivative
    mag = np.hypot(dx, dy)  # magnitude
    mag *= 255 / np.max(mag)  # normalize
    binary_mask = mag >= thrval
    clean_mask = ndimage.morphology.binary_fill_holes(binary_mask).astype(np.uint8)
    return(clean_mask,mag.astype(np.uint8),binary_mask.astype(np.uint8))

In [6]:
def imgRun(img, kernel = 25, threshold_value = threshold_value):
    eimg = exposure.equalize_adapthist(img, kernel_size = kernel)
    bgd = findBackground(eimg).astype(np.uint16)
    bgdImg = np.subtract(eimg, bgd)
    gbgdImg = ndimage.filters.gaussian_filter(bgdImg,1)
    clean_mask,mag_img,dirty_mask = findObjects(gbgdImg,threshold_value)
    super_clean_mask = morphology.binary_closing(clean_mask, footprint=footprint)
    label_img = measure.label(super_clean_mask)
    props = measure.regionprops_table(label_img,intensity_image=img,
                                        properties=['label','area',
                                                    'bbox', 'bbox_area', 'convex_area',
                                                    'centroid', 'weighted_centroid', 'weighted_local_centroid', 
                                                    'eccentricity', 'equivalent_diameter', 
                                                    'inertia_tensor', 'inertia_tensor_eigvals', 
                                                    'local_centroid', 'major_axis_length', 
                                                    'max_intensity', 'mean_intensity', 'min_intensity', 
                                                    'minor_axis_length', 'solidity',  'orientation', 'perimeter', 
                                                    'moments', 'moments_central', 'moments_hu', 
                                                    'moments_normalized', 'weighted_moments', 'weighted_moments_central', 
                                                    'weighted_moments_hu', 'weighted_moments_normalized'])
    return([props,eimg,gbgdImg,super_clean_mask])

In [7]:
dirtouse = askdirectory()

masks = []
readyimgs = []
for root, dirs, files in os.walk(dirtouse):
    if len(files) >0:
        tifs = []
        imgs = []
        for filename in files:
            if filename.endswith('.tif'):
                tifs.append(filename)
                pictouse = os.path.join(root,filename)
                img = io.imread(pictouse,-1)
                imgs.append(img)
        length = len(imgs) - 1

        if length > 0: # check if there are images to test
            allpanalysis = pd.DataFrame()
            #Move through all imgs
            for i in range(len(imgs)):
                print(str(i+1)+'/'+str(length+1))
                img = imgs[i]
                imgname = tifs[i][:-4]
                cimg = img[crop[0]:crop[1],crop[2]:crop[3]]
                thr = threshold_value
                while True:
                    props,eimg,gbgdImg,super_clean_mask = imgRun(cimg, thr) #actually do the analysis
                    panalysis = pd.DataFrame(props)
                    lngth = len(panalysis) #test for the number of objects
                    print(lngth)
                    if thr < 2 or thr > (99-moveval):
                        break
                    if lngth < fewest:
                        if thr < (moveval+1):
                            thr = 1
                        else:
                            thr = thr-moveval
                    elif lngth > most:
                        thr += moveval
                    else:
                        break
                panalysis['total_fluorescence'] = panalysis['area'] * panalysis['mean_intensity']
                panalysis.to_csv(os.path.join(root,imgname+'.csv'))
                panalysis['imagename']= imgname
                allpanalysis = pd.concat([allpanalysis,panalysis.dropna(axis=1)])
            allpanalysis.to_csv(os.path.join(root,'set.csv'))

1/40
51
2/40
50
3/40
9
6
8
25
4/40
10
5/40
26
6/40
8
7
8
21
7/40
8
8
9
13
8/40
4
4
4
12
9/40
13
10/40
23
11/40
14
12/40
18
13/40
27
14/40
62
15/40
12
16/40
14
17/40
13
18/40
8
10
19/40
29
20/40
116
21/40
9
9
9
11
22/40
11
23/40
103
24/40
46
25/40
29
26/40
19
27/40
3
3
3
27
28/40
3
3
3
25
29/40
3
3
4
12
30/40
7
7
8
11
31/40
3
2
2
4
296
32/40
8
8
9
25
33/40
12
34/40
29
35/40
22
36/40
29
37/40
27
38/40
32
39/40
25
40/40
44
